In [2]:
import numpy as np 
import pandas as pd 
from ipynb.fs.full.funciones_FF import feature_RatioDayMoments, feature_Basicos, feature_Basicos_Ult_7d
from ipynb.fs.full.funciones_CF import qty_adoptions

ModuleNotFoundError: No module named 'funciones_FF'

In [2]:
# Para que levante las funciones que se van actualizando en los otros notebooks
%load_ext autoreload
%autoreload 2

In [5]:
# Alternativa (peor) al import ipynb.fs.full
#%run funciones_FF.ipynb
#%run funciones_CF.ipynb

# Seteo del mes snapshot

In [6]:
mes_snapshot = 9  #9 para train 12 para scoreo

# Lectura de los csv

In [7]:
df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_pageviews_complemento = pd.read_csv("./data/pageviews_complemento.csv",
                   parse_dates=["FEC_EVENT"])
df_pageviews = pd.concat([df_pageviews, df_pageviews_complemento],ignore_index=True,sort=False).drop_duplicates().reset_index(drop=True)
df_devicedata = pd.read_csv("./data/device_data.csv",
                   parse_dates=["FEC_EVENT"]).drop_duplicates().reset_index(drop=True)
df_conversiones = pd.read_csv("./data/conversiones.csv").drop_duplicates().reset_index(drop=True)

In [8]:
#Se eliminan los registros que no deben ser considerados.
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month <= mes_snapshot]
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month > mes_snapshot - 9]

df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month <= mes_snapshot]
df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month > mes_snapshot - 9]

# Creación del universo

In [9]:
def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

In [10]:
universo = universo_train(df_pageviews)
universo.head(10)

,user_id,snapshot_mes
0,0,9
1,1,9
2,2,9
3,3,9
4,4,9
5,5,9
6,6,9
7,7,9
8,8,9
9,9,9


# Features

In [7]:
# def feature_X(df):

In [ ]:
#features de pageviews
ratio_page_all(universo, df_pageviews, mes_snapshot)
ratio_ccategory_all(universo, df_pageviews, mes_snapshot)
ratio_cctop_all(universo, df_pageviews, mes_snapshot)
ratio_ccbottom_all(universo, df_pageviews, mes_snapshot)
ratio_site_id_all(universo, df_pageviews, mes_snapshot)
on_site_search_term_user(universo, df_pageviews, mes_snapshot)

In [8]:
#features de device
ratio_mobile_device_all(universo, df_devicedata, mes_snapshot)
ratio_connection_speed_all(universo, df_devicedata, mes_snapshot)

# Armar dataset

In [20]:
# FF methods
universo = feature_RatioDayMoments(universo,df_pageviews)
universo = feature_Basicos(universo,df_pageviews) 
universo = feature_Basicos_Ult_7d(universo,df_pageviews)

# CF methods
universo = qty_adoptions(universo, df_conversiones)

# check df
universo.head(10)              

,user_id,snapshot_mes,target,FEC_EVENT_MAX,CANT_TOTAL_EVER,CANT_TOTAL_ULT_7D,FEC_EVENT_MENOS_7D,RATIO_VISITA_MADRUGADA,RATIO_VISITA_MANANA,RATIO_VISITA_TARDE,...,CONTENT_CATEGORY_BOTTOM_Ult_7d_63,SITE_ID_Ult_7d_1,SITE_ID_Ult_7d_2,SITE_ID_Ult_7d_3,SITE_ID_Ult_7d_4,ON_SITE_SEARCH_TERM_Ult_7d_1,qty_1,qty_3,qty_6,qty_9
0,0,9,0,2018-09-30 20:13:42,4912,90,2018-09-23 20:13:42,0.0,0.466667,0.222222,...,0.0,0.000000,0.244444,0.755556,0.0,1.0,0.0,0.0,0.0,0.0
1,1,9,0,2018-09-30 20:56:50,1634,26,2018-09-23 20:56:50,0.0,0.961538,0.000000,...,0.0,0.115385,0.230769,0.653846,0.0,1.0,0.0,0.0,0.0,0.0
2,2,9,0,2018-09-28 00:04:58,1717,15,2018-09-21 00:04:58,0.2,0.466667,0.333333,...,0.0,0.200000,0.266667,0.533333,0.0,1.0,0.0,0.0,0.0,0.0
3,3,9,0,2018-09-28 16:03:37,1554,61,2018-09-21 16:03:37,0.0,0.049180,0.950820,...,0.0,0.016393,0.229508,0.754098,0.0,1.0,0.0,0.0,0.0,0.0
4,4,9,0,2018-09-27 22:15:33,2478,50,2018-09-20 22:15:33,0.0,0.240000,0.300000,...,0.0,0.000000,0.420000,0.580000,0.0,1.0,0.0,0.0,0.0,0.0
5,5,9,0,2018-09-30 13:13:39,4250,72,2018-09-23 13:13:39,0.0,0.361111,0.486111,...,0.0,0.041667,0.111111,0.847222,0.0,1.0,0.0,0.0,0.0,0.0
6,6,9,0,2018-09-28 15:36:45,1108,48,2018-09-21 15:36:45,0.0,0.395833,0.604167,...,0.0,0.000000,0.145833,0.854167,0.0,1.0,0.0,0.0,0.0,0.0
7,7,9,0,2018-09-27 09:19:27,1793,11,2018-09-20 09:19:27,0.0,0.272727,0.727273,...,0.0,0.000000,0.363636,0.636364,0.0,1.0,0.0,0.0,0.0,0.0
8,8,9,0,2018-09-26 12:41:21,1549,137,2018-09-19 12:41:21,0.0,0.000000,0.744526,...,0.0,0.080292,0.131387,0.788321,0.0,1.0,0.0,0.0,0.0,0.0
9,9,9,0,2018-09-30 22:01:01,1591,40,2018-09-23 22:01:01,0.0,0.000000,0.325000,...,0.0,0.050000,0.125000,0.825000,0.0,1.0,0.0,0.0,0.0,0.0


# Target

In [11]:
def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 
    
    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()

In [12]:
target_universo = target(universo)
universo = pd.merge(universo, target_universo, how='left', left_on=['user_id', 'snapshot_mes'], right_on=['user_id','snapshot_mes'])

In [15]:
universo.to_csv(r'dataset.csv', index = None, header=True)

In [ ]:
df_pageviews.to_csv(r'df_pageviews.csv', index = None, header=True)

In [ ]:
df_conversiones.to_csv(r'df_conversiones.csv', index = None, header=True)